# 02 - Análisis Exploratorio de Datos (EDA)

Análisis profundo de los datos de Airbnb Madrid usando MongoDB y Pandas.

## Contenido
1. Carga de datos
2. Análisis de precios
3. Análisis geográfico
4. Análisis de reviews
5. Análisis de hosts
6. Correlaciones

In [ ]:
# Imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from src.crud_operations import AirbnbCRUD
from src.database import MongoDBConnection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Imports completados")

## 1. Carga de Datos

In [ ]:
# Conectar y cargar datos
crud = AirbnbCRUD()
conn = MongoDBConnection()

print("Cargando datos...")
listings = list(crud.collection.find())  # Cargar todos
df = pd.DataFrame(listings)

print(f"✅ Datos cargados: {df.shape[0]:,} listings con {df.shape[1]} columnas")

In [ ]:
# Información general del dataset
print("📊 INFORMACIÓN GENERAL:\n")
print(df.info())

In [ ]:
# Vista previa
display(df.head())

## 2. Análisis de Precios

In [ ]:
# Estadísticas de precio
if 'price' in df.columns:
    print("💰 ESTADÍSTICAS DE PRECIO:\n")
    print(df['price'].describe())
    print(f"\nRango: {df['price'].min():.2f}€ - {df['price'].max():.2f}€")
    print(f"Mediana: {df['price'].median():.2f}€")
    print(f"Moda: {df['price'].mode()[0]:.2f}€")

In [ ]:
# Distribución de precios
if 'price' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histograma
    axes[0].hist(df[df['price'] < 500]['price'], bins=50, edgecolor='black')
    axes[0].set_title('Distribución de Precios (< 500€)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Precio (€)')
    axes[0].set_ylabel('Frecuencia')
    axes[0].axvline(df['price'].median(), color='red', linestyle='--', label=f'Mediana: {df["price"].median():.2f}€')
    axes[0].legend()
    
    # Boxplot
    axes[1].boxplot(df[df['price'] < 500]['price'])
    axes[1].set_title('Boxplot de Precios (< 500€)', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Precio (€)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Precio por tipo de habitación
if 'price' in df.columns and 'room_type' in df.columns:
    precio_por_tipo = df.groupby('room_type')['price'].agg(['mean', 'median', 'count']).round(2)
    precio_por_tipo = precio_por_tipo.sort_values('mean', ascending=False)
    
    print("\n🏠 PRECIO POR TIPO DE HABITACIÓN:\n")
    display(precio_por_tipo)
    
    # Gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    precio_por_tipo['mean'].plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
    ax.set_title('Precio Promedio por Tipo de Habitación', fontsize=14, fontweight='bold')
    ax.set_xlabel('Tipo de Habitación')
    ax.set_ylabel('Precio Promedio (€)')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 3. Análisis Geográfico

In [ ]:
# Listings por barrio
if 'neighbourhood_cleansed' in df.columns:
    barrios = df['neighbourhood_cleansed'].value_counts().head(15)
    
    print("📍 TOP 15 BARRIOS CON MÁS LISTINGS:\n")
    for i, (barrio, count) in enumerate(barrios.items(), 1):
        pct = count / len(df) * 100
        print(f"{i:2d}. {barrio:30s}: {count:5,} ({pct:5.2f}%)")
    
    # Gráfico
    fig, ax = plt.subplots(figsize=(12, 6))
    barrios.plot(kind='barh', ax=ax, color='coral', edgecolor='black')
    ax.set_title('Top 15 Barrios con Más Listings', fontsize=14, fontweight='bold')
    ax.set_xlabel('Número de Listings')
    ax.set_ylabel('Barrio')
    plt.tight_layout()
    plt.show()

In [ ]:
# Precio promedio por barrio (top 15)
if 'price' in df.columns and 'neighbourhood_cleansed' in df.columns:
    precio_barrio = df.groupby('neighbourhood_cleansed')['price'].mean().sort_values(ascending=False).head(15)
    
    print("\n💰 TOP 15 BARRIOS MÁS CAROS:\n")
    for i, (barrio, precio) in enumerate(precio_barrio.items(), 1):
        print(f"{i:2d}. {barrio:30s}: {precio:7.2f}€")
    
    # Gráfico
    fig, ax = plt.subplots(figsize=(12, 6))
    precio_barrio.plot(kind='barh', ax=ax, color='gold', edgecolor='black')
    ax.set_title('Top 15 Barrios Más Caros (Precio Promedio)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Precio Promedio (€)')
    ax.set_ylabel('Barrio')
    plt.tight_layout()
    plt.show()

## 4. Análisis de Reviews

In [ ]:
# Estadísticas de reviews
if 'number_of_reviews' in df.columns:
    print("⭐ ESTADÍSTICAS DE REVIEWS:\n")
    print(df['number_of_reviews'].describe())
    
    # Distribución
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histograma
    axes[0].hist(df[df['number_of_reviews'] < 100]['number_of_reviews'], bins=50, edgecolor='black', color='lightgreen')
    axes[0].set_title('Distribución de Reviews (< 100)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Número de Reviews')
    axes[0].set_ylabel('Frecuencia')
    
    # Boxplot
    axes[1].boxplot(df[df['number_of_reviews'] < 100]['number_of_reviews'])
    axes[1].set_title('Boxplot de Reviews (< 100)', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Número de Reviews')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Relación precio vs reviews
if 'price' in df.columns and 'number_of_reviews' in df.columns:
    # Filtrar outliers para mejor visualización
    df_filtered = df[(df['price'] < 300) & (df['number_of_reviews'] < 200)]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(df_filtered['number_of_reviews'], df_filtered['price'], alpha=0.5)
    ax.set_title('Relación: Número de Reviews vs Precio', fontsize=14, fontweight='bold')
    ax.set_xlabel('Número de Reviews')
    ax.set_ylabel('Precio (€)')
    plt.tight_layout()
    plt.show()
    
    # Correlación
    corr = df[['price', 'number_of_reviews']].corr().iloc[0, 1]
    print(f"\n📊 Correlación precio-reviews: {corr:.3f}")

## 5. Análisis de Hosts

In [ ]:
# Hosts con más listings
if 'host_id' in df.columns and 'host_name' in df.columns:
    host_listings = df.groupby(['host_id', 'host_name']).size().sort_values(ascending=False).head(15)
    
    print("👤 TOP 15 HOSTS CON MÁS LISTINGS:\n")
    for i, ((host_id, host_name), count) in enumerate(host_listings.items(), 1):
        print(f"{i:2d}. {host_name:30s}: {count:3d} listings")
    
    # Gráfico
    fig, ax = plt.subplots(figsize=(12, 6))
    host_listings.reset_index()['size'].plot(kind='bar', ax=ax, color='mediumpurple', edgecolor='black')
    ax.set_title('Top 15 Hosts con Más Listings', fontsize=14, fontweight='bold')
    ax.set_xlabel('Host')
    ax.set_ylabel('Número de Listings')
    ax.set_xticklabels([h[1][:20] for h in host_listings.index], rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 6. Matriz de Correlación

In [ ]:
# Seleccionar columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cols_importantes = ['price', 'number_of_reviews', 'accommodates', 'bedrooms', 'beds', 
                   'availability_30', 'availability_365']
cols_disponibles = [col for col in cols_importantes if col in numeric_cols]

if len(cols_disponibles) >= 2:
    # Matriz de correlación
    corr_matrix = df[cols_disponibles].corr()
    
    # Visualización
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, ax=ax, cbar_kws={"shrink": 0.8})
    ax.set_title('Matriz de Correlación', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 🎉 Conclusiones

En este notebook realizamos:

- ✅ Análisis de distribución de precios
- ✅ Análisis geográfico por barrios
- ✅ Análisis de reviews
- ✅ Análisis de hosts
- ✅ Correlaciones entre variables

**Siguiente:** `03_visualizations.ipynb` - Visualizaciones avanzadas